# QUESTIONS ARE ANSWERED IN THE DOCUMENT

In [28]:
#importing libraries 

import urllib

from PIL import Image
from torchvision import transforms

from torchvision import transforms
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import os
import random
import cv2


In [9]:
import torch

model1 = torch.hub.load('pytorch/vision:v0.10.0', 'alexnet', pretrained=True) #loading alexnet
model1.eval()

model2 = torch.hub.load('pytorch/vision:v0.10.1', 'vgg16', pretrained=True) #loading vgg16
model2.eval()

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0
Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.1


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [10]:
# Download an example image from the pytorch website
import urllib
url, filename = ("https://github.com/pytorch/hub/raw/master/images/dog.jpg", "dog.jpg")
try: urllib.URLopener().retrieve(url, filename)
except: urllib.request.urlretrieve(url, filename)



In [27]:
dir_list = os.listdir('/content/drive/MyDrive/Colab Notebooks/fruits_images') # https://www.geeksforgeeks.org/python-os-listdir-method/

alex, vgg = [], []

for filename in dir_list:
    input_image = Image.open(filename)

    process = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    input_tensor = process(input_image)
    input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model

    input_batch = process(os.path.join("fruits_images",filename))
    # move the input and model to GPU for speed if available
    if torch.cuda.is_available():
        input_batch = input_batch.to('cuda')
        model1.to('cuda')
    with torch.no_grad():
        alex_output = model1(input_batch)
        vgg_output = model2(input_batch)
    # Tensor of shape 1000, with confidence scores over Imagenet's 1000 classes
    # The output has unnormalized scores. To get probabilities, you can run a softmax on it.
    alex.append(torch.nn.functional.softmax(alex_output[0], dim=0))
    vgg.append(torch.nn.functional.softmax(vgg_output[0], dim=0))
    p_alex = torch.max(alex_output.data,1)
    p_vgg = torch.max(vgg_output.data,1)

for j in range(len(dir_list)):
    # Read the categories
    with open("imagenet_classes.txt", "r") as f:
        categories = [s.strip() for s in f.readlines()]
    # Show top categories per image
    top5_prob_alex, top5_catid_alex = torch.topk(p_alex[j], 1)
    top5_prob_vgg, top5_catid_vgg = torch.topk(p_vgg[j], 1)
    filename = Image.open(os.path.join("Classifiction_Images",dir_list[j]))
    plt.imshow(filename)
    plt.show()
    print("Alexnet Prediction: ",categories[top5_catid_alex[0]], top5_prob_alex[0].item())
    print("VGG Prediction: ",categories[top5_catid_vgg[0]], top5_prob_vgg[0].item())


FileNotFoundError: ignored

In [29]:
#function to add noise

img = cv2.imread(os.path.join("fruits_images",dir_list[1]))

pixelsNum = random.randint(300, 50000)

for i in range(pixelsNum):  
    temp_x = np.random.randint(0, img.shape[0])

    temp_y = np.random.randint(0, img.shape[1])

    img[temp_x][temp_y] = 0

for i in range(pixelsNum):  

    temp_x = np.random.randint(0, img.shape[0])

    temp_y = np.random.randint(0, img.shape[1])

    img[temp_x][temp_y] = 255


cv2.imwrite("Adversarial_image.jpg",img)

  


AttributeError: ignored